Задача:

Мы запустили А/В тест длительностью 10 дней.На каждую метрику направили по 10 000 юзеров. Ошибка первого рода 0.05. На основе выбранных данных, выберите 2 A/B-теста и составьте план эксперимента, чтобы избежать ошибки Peeking Problem

Ключевые метрики:

Количество новых регистраций пользователей

Доход от новых пользователей

Гипотеза: Если предложить 10% скидку на первую покупку для новых пользователей, то мы сможем повысить количество регистраций новых пользователей на 15%.

Что мы делаем в каждой из групп: Даём код на 10% скидку на первую покупку.

На каких пользователях мы тестируем: Новые пользователи, которые раньше не делали покупок на сайте.

Метрика: количество новых регистраций пользователей.

Гипотеза: Если предоставить персонализированные рекомендации по продуктам новым пользователям на основе их истории просмотров и покупательского поведения, то сможем повысить показатель дохода от новых пользователей на n%.
Что мы делаем в каждой из групп: группа новых пользователей получает персонализированные рекомендации.

На каких пользователях мы тестируем: На новых пользователях, совершивших хотя бы одну покупку на сайте

Метрика: доход от новых пользователей.
    
Решение:

Соберём данные о регистрациях и доходах для контрольной и тестовой групп.

Разделим данные на выборки – каждая выборка будет содержать 10,000 пользователей.

Для каждой выборки будем считать количество новых регистраций и доход от новых пользователей для контрольной и тестовой групп.
С помощью бутстрапа будем генерировать 1000 выборок с возвращением.

Для каждой выборки будем считать различие между контрольной и тестовой групп в количестве новых регистраций и доходе от новых пользователей.

Построим распределение полученных различий, оценим доверительные интервалы для обеих метрик.

Если доверительный интервал не содержит 0 (для метрики регистраций) или отрицательных значений (для метрики дохода), то можем сделать вывод о том, что эксперимент был успешным и можно внедрять новые подходы в работу сервиса.

Определяем ключевые билиотеки:    

    
    

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm

Собираем данные:

In [3]:
control_registrations = 10000
test_registrations = 10000
control_income = 1000000
test_income = 1000000

Разделяем данные на выборки:

In [4]:
control_registrations_list = np.random.binomial(n=1, p=0.05, size=control_registrations)
test_registrations_list = np.random.binomial(n=1, p=0.05, size=test_registrations)
control_income_list = np.random.normal(loc=100, scale=20, size=control_income)
test_income_list = np.random.normal(loc=110, scale=20, size=test_income)

Считаем количество регистраций и доход для контрольной и тестовой групп:

In [5]:
control_registrations_num = sum(control_registrations_list)
test_registrations_num = sum(test_registrations_list)
control_income_sum = sum(control_income_list)
test_income_sum = sum(test_income_list)

Проводим бутстрап:

In [6]:
n = 1000
differences_registrations = []
differences_income = []

for i in range(n):
    control_registrations_sample = np.random.choice(control_registrations_list, size=10000, replace=True)
    test_registrations_sample = np.random.choice(test_registrations_list, size=10000, replace=True)
    control_income_sample = np.random.choice(control_income_list, size=10000, replace=True)
    test_income_sample = np.random.choice(test_income_list, size=10000, replace=True)

    sample_diff_registrations = sum(test_registrations_sample) - sum(control_registrations_sample)
    sample_diff_income = sum(test_income_sample) - sum(control_income_sample)

    differences_registrations.append(sample_diff_registrations)
    differences_income.append(sample_diff_income)

Оцениваем доверительный интервал для количества регистраций:

In [7]:
difference_registrations_mean = np.mean(differences_registrations)
difference_registrations_std = np.std(differences_registrations)
ci_registrations = norm.interval(0.95, loc=difference_registrations_mean, scale=difference_registrations_std/np.sqrt(n))
print("95% доверительный интервал для количества регистраций: ", ci_registrations)

95% доверительный интервал для количества регистраций:  (16.50440920475523, 20.22159079524477)


Оцениваем доверительный интервал для дохода:

In [8]:
difference_income_mean = np.mean(differences_income)
difference_income_std = np.std(differences_income)
ci_income = norm.interval(0.95, loc=difference_income_mean, scale=difference_income_std/np.sqrt(n))
print("95% доверительный интервал для дохода: ", ci_income)

95% доверительный интервал для дохода:  (99812.69012237685, 100160.6596084094)


Ответ:

Так как, доверительный интервал не содержит 0 (для метрики регистраций) и отрицательных значений (для метрики дохода), то можем сделать вывод о том, что эксперимент был успешным и можно внедрять новые подходы в работу сервиса.